# 02. Naive RAG 실패 분석 - 파스텔 멀티홉 시나리오 (60분)

## 🎯 학습 목표
- **동음이의어**와 **멀티홉 추론**에서 Naive RAG가 실패하는 이유 체험
- 실제 사용자 질문으로 RAG 한계점을 직관적으로 이해
- Advanced RAG 기법의 필요성을 구체적으로 체감

## 📋 실습 구성
1. **파스텔 멀티홉 코퍼스 소개** (5분)
2. **나이브 RAG 시스템 구축** (10분) 
3. **핵심 실패 시나리오 테스트** (15분)
4. **정답과 실패 원인 상세 분석** (20분)
5. **핵심 교훈 및 Advanced RAG 예고** (10분)

---

> 💡 **핵심 아이디어**: "파스텔"이라는 하나의 단어가 어떻게 RAG 시스템을 혼란스럽게 만드는지 체험해보세요!

In [ ]:
# 필수 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers matplotlib seaborn pandas numpy

import os
import time
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# LangChain 관련 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

# Transformers 관련 import  
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pydantic import Field

# 한글 폰트 설정
import matplotlib.font_manager as fm
import platform

if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = ['AppleGothic']
elif platform.system() == 'Windows':  # Windows
    plt.rcParams['font.family'] = ['Malgun Gothic']
else:  # Linux/Colab
    plt.rcParams['font.family'] = ['NanumGothic', 'DejaVu Sans']

plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 및 폰트 설정 완료!")

## 1. 파스텔 멀티홉 코퍼스 소개 (5분)

### 🍰 왜 "파스텔"인가?
실제 사용자들은 명확하지 않은 질문을 던집니다. "파스텔"이라고 하면 호텔인지, 카페인지, 소극장인지 명확하지 않죠. 이런 **동음이의어 문제**가 RAG 시스템의 첫 번째 난관입니다.

### 🔗 멀티홉 추론이란?
하나의 답변을 위해 **여러 문서의 정보를 연결**해야 하는 상황입니다.
- "파스텔 예약 변경" → 호텔 문서 필요
- "덜 붐비는 미술관 시간" → 미술관 문서 필요  
- 두 정보를 **종합**해서 답변 → 멀티홉 추론

이제 실제 데이터를 살펴보겠습니다! 🚀

In [ ]:
# Day 1 파인튜닝 모델 클래스 (깔끔하게 정리)
class Day1FinetunedLLM(LLM):
    """Day 1 파인튜닝 모델을 사용하는 LLM 클래스"""
    
    # Pydantic 필드 선언
    model_name: str = Field(default="ryanu/my-exaone-raft-model")
    tokenizer: Any = Field(default=None)
    model: Any = Field(default=None)
    
    class Config:
        arbitrary_types_allowed = True
    
    def __init__(self, model_name: str = "ryanu/my-exaone-raft-model", **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        print(f"🎯 Day 1 파인튜닝 모델 로드: {self.model_name}")
        self._load_model()
    
    def _load_model(self):
        """모델 로드"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        print("✅ 모델 로드 완료!")
    
    @property
    def _llm_type(self) -> str:
        return "day1_finetuned_llm"
    
    def _call(self, prompt: str, stop=None, run_manager=None, **kwargs) -> str:
        """실제 모델 추론"""
        # EXAONE 프롬프트 템플릿 적용
        formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
        
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs, max_new_tokens=512, temperature=0.7, do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id, eos_token_id=self.tokenizer.eos_token_id
            )
        
        response = self.tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        return response if response else "적절한 답변을 생성할 수 없습니다."

print("✅ Day1FinetunedLLM 클래스 정의 완료!")

## 2. 나이브 RAG 시스템 구축 (10분)

### 🛠️ 시스템 설정
- **STRICT 프롬프트**: LLM의 일반지식을 완전 차단하여 진짜 RAG 실패를 보여줍니다
- **작은 청크**: 200자로 의도적으로 작게 설정하여 컨텍스트 파편화 유도  
- **제한적 검색**: Top-2만 검색하여 정보 부족 상황 연출

이제 파스텔 코퍼스와 RAG 시스템을 구축해보겠습니다!

In [ ]:
# 🍰 파스텔 멀티홉 코퍼스 (간단하고 직관적!)
def create_pastel_corpus():
    """파스텔 동음이의어로 멀티홉 추론 실패를 보여줄 재미있는 문서들"""
    documents = [
        {
            "content": """파스텔 호텔 예약 안내

서울 강남구에 위치한 파스텔 호텔입니다.
- 주소: 서울특별시 강남구 테헤란로 123
- 체크인: 15:00, 체크아웃: 11:00
- 예약 변경: 투숙 3일 전까지 가능 (위약금 없음)
- 주말 요금: 평일 대비 30% 할인
- 문의: 02-1234-5678

특별 서비스:
- 조식 뷔페 운영 (07:00-10:00)
- 피트니스 센터 24시간 이용 가능""",
            "metadata": {"type": "hotel", "name": "파스텔", "location": "강남"}
        },
        {
            "content": """파스텔 카페 메뉴 및 운영시간

홍대 파스텔 카페는 아기자기한 디저트로 유명합니다.
- 주소: 서울특별시 마포구 홍익로 456
- 운영시간: 09:00-22:00 (연중무휴)
- 시그니처 메뉴: 파스텔 마카롱, 레인보우 케이크
- 예약: 단체 예약만 가능 (10명 이상)
- 문의: 02-9876-5432

주말 특별 이벤트:
- 토요일: 마카롱 만들기 체험 (14:00-16:00)
- 일요일: 케이크 데코레이션 클래스 (15:00-17:00)""",
            "metadata": {"type": "cafe", "name": "파스텔", "location": "홍대"}
        },
        {
            "content": """파스텔 소극장 공연 안내

대학로 파스텔 소극장에서 뮤지컬을 상영합니다.
- 주소: 서울특별시 종로구 대학로 789
- 현재 공연: '꿈꾸는 파스텔' 뮤지컬
- 공연시간: 화-일 19:30 (월요일 휴관)
- 티켓 예약: 인터파크, 현장 구매 가능
- 관람료: 일반 30,000원, 학생 20,000원

주말 특별 공연:
- 토요일: 15:00 추가 공연 (가족 할인 적용)
- 일요일: 17:00 브런치 공연 (음료 서비스 포함)""",
            "metadata": {"type": "theater", "name": "파스텔", "location": "대학로"}
        },
        {
            "content": """국립현대미술관 전시 정보

서울 국립현대미술관 덕수궁관 전시 안내입니다.
- 주소: 서울특별시 중구 세종대로 99
- 운영시간: 10:00-19:00 (월요일 휴관)
- 현재 전시: '색채의 여행' 기획전
- 입장료: 성인 4,000원, 청소년 2,000원

주말 관람 팁:
- 토요일: 상대적으로 붐빔 (14:00-16:00 피크)
- 일요일: 오전이 한적함 (10:00-12:00 추천)
- 도슨트 투어: 11:00, 15:00 (주말 한정)""",
            "metadata": {"type": "museum", "name": "국립현대미술관", "location": "중구"}
        },
        {
            "content": """서울 여행 블로그 - 핫플 추천

서울 여행 중 꼭 가볼 만한 숨은 명소들을 소개합니다!

강남 투어:
- 파스텔 같은 감성 숍들이 많아요
- 쇼핑과 맛집이 집중된 지역
- 교통이 편리해서 접근성 좋음

문화 체험:
- 미술관은 주말 오전이 베스트 타이밍
- 덜 붐비는 시간대를 노리면 여유롭게 관람 가능
- 사진 촬영도 자유로운 편

여행 꿀팁:
- 예약은 미리미리! 특히 숙소는 필수
- 주말 할인료 체크하고 예산 계획 세우기""",
            "metadata": {"type": "blog", "category": "travel", "focus": "seoul_tips"}
        }
    ]
    return documents

print("✅ 파스텔 멀티홉 코퍼스 준비 완료!")

In [ ]:
# ⚡ STRICT 프롬프트로 파스텔 나이브 RAG 구현
from langchain.prompts import PromptTemplate
from langchain.schema import Document

# 🔒 LLM 일반지식 완전 차단 프롬프트
STRICT_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "아래 컨텍스트에 **있는 내용만** 근거로 한국어로 답하세요.\n"
        "- 컨텍스트에 없는 정보는 **절대** 추측/일반지식으로 보완하지 마세요.\n"
        "- 근거가 부족하면: '근거 불충분: ○○ 정보 필요'라고 한 줄로 말하고, "
        "추가로 필요한 정보 1~2가지만 물어보세요.\n\n"
        "컨텍스트:\n{context}\n\n질문:\n{question}\n\n답변:"
    )
)

def setup_pastel_naive_rag():
    """🍰 파스텔 코퍼스로 나이브 RAG 시스템 구축 (일반지식 차단)"""
    print("🔧 파스텔 나이브 RAG 시스템 구축 중...")
    
    # 파스텔 문서 준비
    documents = create_pastel_corpus()
    langchain_docs = [Document(page_content=doc['content'], metadata=doc['metadata']) 
                     for doc in documents]
    
    # 작은 청크로 분할 (실패 유도)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    split_docs = text_splitter.split_documents(langchain_docs)
    
    # 벡터스토어 생성
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    # LLM 로드
    llm = Day1FinetunedLLM()
    
    # ⭐ 더 제한적인 Top-2 검색으로 실패 유도
    naive_rag_strict = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
        chain_type_kwargs={"prompt": STRICT_PROMPT},
        return_source_documents=True,
    )
    
    print(f"✅ 파스텔 나이브 RAG 시스템 준비 완료!")
    print(f"   • 총 청크: {len(split_docs)}개")
    print(f"   • 청크 크기: 200자 (의도적으로 작게)")
    print(f"   • 제약: LLM 일반지식 완전 차단")
    print(f"   • 검색: 단순 벡터 유사도 Top-2만 (더 제한적!)")
    
    return naive_rag_strict

# 파스텔 나이브 RAG 시스템 초기화
naive_rag = setup_pastel_naive_rag()

In [ ]:
# 🍰 핵심 실패 시나리오 테스트
def test_single_pastel_scenario():
    """핵심 파스텔 시나리오 하나만 집중 테스트"""
    
    # 테스트할 핵심 시나리오
    scenario = {
        "query": "서울 가는데 파스텔 예약 변경하고, 주말에 덜 붐비는 미술관 시간도 추천해줘",
        "description": "🔗 동음이의어 + 멀티홉 추론",
        "challenges": [
            "'파스텔'이 호텔/카페/소극장 중 무엇인지 불분명",
            "예약 변경 + 미술관 추천 두 정보를 연결해야 함",
            "'서울 가는데'라는 맥락으로 호텔임을 추론해야 함"
        ]
    }
    
    print("🧪 파스텔 멀티홉 실패 시나리오 테스트")
    print("="*60)
    print("💡 실제 사용자가 던질 법한 질문으로 나이브 RAG의 한계를 체험해보세요!")
    print()
    print(f"🎯 테스트 시나리오: {scenario['description']}")
    for i, challenge in enumerate(scenario['challenges'], 1):
        print(f"   {i}. {challenge}")
    print()
    
    # 실제 쿼리 실행
    print(f"❓ **사용자 질문**: {scenario['query']}")
    print()
    print("🤖 **나이브 RAG 답변**:")
    
    start_time = time.time()
    result = naive_rag({"query": scenario['query']})
    end_time = time.time()
    
    # 결과 분석
    answer = result.get('result', '')
    retrieved_docs = result.get('source_documents', [])
    response_time = end_time - start_time
    
    print(f"'{answer}'")
    print()
    print(f"⏱️ 응답시간: {response_time:.2f}초")
    print(f"📄 검색된 문서: {len(retrieved_docs)}개")
    
    # 검색된 문서 상세 표시
    if retrieved_docs:
        print()
        print("🔍 **검색된 문서 내용**:")
        for i, doc in enumerate(retrieved_docs, 1):
            doc_type = doc.metadata.get('type', 'unknown')
            location = doc.metadata.get('location', 'unknown')
            content_preview = doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content
            print(f"   **{i}. [{doc_type}-{location}]**")
            print(f"   {content_preview}")
            print()
    
    # 실패 판정
    is_failed = (
        "근거 불충분" in answer or 
        "정보 필요" in answer or 
        len(answer.strip()) < 50 or
        "모르겠습니다" in answer or
        "답변할 수 없습니다" in answer
    )
    
    status = "❌ **실패!**" if is_failed else "⚠️ **부분 성공**"
    print(f"📊 **최종 결과**: {status}")
    
    return {
        'scenario': scenario,
        'answer': answer,
        'is_failed': is_failed,
        'response_time': response_time,
        'retrieved_docs': retrieved_docs
    }

# 핵심 시나리오 실행
print("## 3. 핵심 실패 시나리오 테스트 (15분)")
print()
print("### 🎯 테스트 시나리오")
print("**질문**: \"서울 가는데 파스텔 예약 변경하고, 주말에 덜 붐비는 미술관 시간도 추천해줘\"")
print()
print("이 질문이 어려운 이유:")
print("- 🤷 **동음이의어**: \"파스텔\"이 호텔/카페/소극장 중 무엇인지 모호")
print("- 🧩 **멀티홉**: 예약 변경 + 미술관 추천 두 가지 정보 연결 필요")
print("- 📍 **맥락 추론**: \"서울 가는데\"라는 맥락으로 호텔임을 추측해야 함")
print()
print("과연 나이브 RAG가 이 복합적인 질문을 제대로 처리할 수 있을까요?")
print()

test_result = test_single_pastel_scenario()

In [ ]:
## 4. 정답과 실패 원인 상세 분석 (20분)

### ✅ 올바른 답변 (원본 문서 기반)

def show_correct_answer():
    """원본 문서를 바탕으로 올바른 답변 제시"""
    print("### ✅ **올바른 답변**")
    print("---")
    print()
    print("**'파스텔'은 호텔로 보는 게 자연스럽습니다** ('예약 변경' 규정이 문서에 있음).")
    print()
    print("**📋 예약 변경 안내:**")
    print("- 투숙 3일 전까지 가능")
    print("- 위약금 없음") 
    print()
    print("**🎨 미술관 추천:**")
    print("- **추천**: 일요일 오전 10:00–12:00 (한적함)")
    print("- **피하세요**: 토요일 14:00–16:00 (혼잡)")
    print()
    print("**💡 추론 과정:**")
    print("1. '서울 가는데' + '예약 변경' → 숙박시설(호텔) 의미")
    print("2. 파스텔 호텔 문서에서 예약 변경 규정 확인")
    print("3. 미술관 문서에서 주말 관람 팁 확인")
    print("4. 두 정보를 종합하여 완전한 답변 제공")
    print()
    print("---")
    print("**💭 참고사항:**")
    print("- 만약 '파스텔 카페(홍대)'를 말한 거라면: **단체(10명 이상)만** 예약 가능 → 일반 예약 변경은 해당 없음")
    print("- '파스텔 소극장'은 공연 티켓 규정이라 숙소와 무관")

### 🔍 나이브 RAG는 왜 실패했을까요?

def analyze_failure_detailed(result):
    """나이브 RAG 실패 원인을 단계별로 상세 분석"""
    print()
    print("### 🔍 **실패 원인 단계별 분석**")
    print("---")
    
    print()
    print("#### 1️⃣ **검색 단계 실패**")
    retrieved_docs = result['retrieved_docs']
    if retrieved_docs:
        print("**실제 검색된 문서:**")
        for i, doc in enumerate(retrieved_docs, 1):
            doc_type = doc.metadata.get('type', 'unknown') 
            location = doc.metadata.get('location', 'unknown')
            print(f"- {doc_type} ({location})")
        print()
        print("**문제점:**")
        print("- Top-2 제한으로 필요한 문서들을 모두 검색하지 못함")
        print("- '파스텔' 키워드로만 검색하여 미술관 정보 누락 가능성")
        print("- 벡터 유사도만으로는 멀티홉 질문의 복잡성 처리 불가")
    
    print()
    print("#### 2️⃣ **맥락 이해 실패**") 
    print("**나이브 RAG의 한계:**")
    print("- '서울 가는데'라는 맥락으로 호텔임을 추론하지 못함")
    print("- '예약 변경'이라는 단서를 활용하지 못함")
    print("- 동음이의어 '파스텔' 중 어느 것인지 판단 불가")
    
    print()
    print("#### 3️⃣ **정보 연결 실패**")
    print("**멀티홉 추론 부족:**")
    print("- 파스텔 호텔 정보 + 미술관 정보를 독립적으로 처리")
    print("- 두 정보를 종합한 통합 답변 생성 불가")
    print("- 단순 검색 결과 나열에 그침")
    
    print()
    print("#### 4️⃣ **STRICT 프롬프트의 영향**")
    print("**일반지식 차단 효과:**")
    print("- LLM의 배경지식 활용 불가")
    print("- 컨텍스트에 없는 정보는 절대 추론하지 않음")
    print("- 진짜 RAG 시스템의 한계가 그대로 노출됨")

### 📊 나이브 RAG vs 사람의 추론 과정 비교

def compare_reasoning_process():
    """나이브 RAG와 사람의 추론 과정 비교"""
    print()
    print("### 📊 **추론 과정 비교**")
    print("---")
    print()
    
    print("| 단계 | 👤 **사람의 추론** | 🤖 **나이브 RAG** |")
    print("|------|-------------------|-------------------|")
    print("| **1. 질문 이해** | '서울 가는데' → 여행/숙박 상황<br/>'예약 변경' → 호텔 가능성 높음 | '파스텔' 키워드만 추출<br/>맥락 무시 |")
    print("| **2. 엔티티 해결** | 파스텔 = 호텔로 판단<br/>(맥락 기반 추론) | 파스텔 = 모호함<br/>(동음이의어 미해결) |")
    print("| **3. 정보 수집** | 호텔 정보 + 미술관 정보<br/>둘 다 필요하다고 판단 | 유사도 높은 문서 2개만<br/>편향적 검색 |")
    print("| **4. 답변 생성** | 두 정보를 종합하여<br/>완전한 답변 제공 | 검색된 정보만으로<br/>부분적 답변 |")
    
    print()
    print("**🎯 핵심 차이점:**")
    print("- **사람**: 맥락을 이해하고 추론하여 정보를 연결")  
    print("- **나이브 RAG**: 단순 키워드 매칭과 유사도 검색에 의존")

# 분석 함수들 실행
show_correct_answer()
analyze_failure_detailed(test_result)
compare_reasoning_process()

In [ ]:
## 4. 정답과 실패 원인 상세 분석 (20분)

### ✅ 올바른 답변 (원본 문서 기반)

def show_correct_answer():
    """원본 문서를 바탕으로 올바른 답변 제시"""
    print("### ✅ **올바른 답변**")
    print("---")
    print()
    print("**'파스텔'은 호텔로 보는 게 자연스럽습니다** ('예약 변경' 규정이 문서에 있음).")
    print()
    print("**📋 예약 변경 안내:**")
    print("- 투숙 3일 전까지 가능")
    print("- 위약금 없음") 
    print()
    print("**🎨 미술관 추천:**")
    print("- **추천**: 일요일 오전 10:00–12:00 (한적함)")
    print("- **피하세요**: 토요일 14:00–16:00 (혼잡)")
    print()
    print("**💡 추론 과정:**")
    print("1. '서울 가는데' + '예약 변경' → 숙박시설(호텔) 의미")
    print("2. 파스텔 호텔 문서에서 예약 변경 규정 확인")
    print("3. 미술관 문서에서 주말 관람 팁 확인")
    print("4. 두 정보를 종합하여 완전한 답변 제공")
    print()
    print("---")
    print("**💭 참고사항:**")
    print("- 만약 '파스텔 카페(홍대)'를 말한 거라면: **단체(10명 이상)만** 예약 가능 → 일반 예약 변경은 해당 없음")
    print("- '파스텔 소극장'은 공연 티켓 규정이라 숙소와 무관")

### 🔍 나이브 RAG는 왜 실패했을까요?

def analyze_failure_detailed(result):
    """나이브 RAG 실패 원인을 단계별로 상세 분석"""
    print()
    print("### 🔍 **실패 원인 단계별 분석**")
    print("---")
    
    print()
    print("#### 1️⃣ **검색 단계 실패**")
    retrieved_docs = result['retrieved_docs']
    if retrieved_docs:
        print("**실제 검색된 문서:**")
        for i, doc in enumerate(retrieved_docs, 1):
            doc_type = doc.metadata.get('type', 'unknown') 
            location = doc.metadata.get('location', 'unknown')
            print(f"- {doc_type} ({location})")
        print()
        print("**문제점:**")
        print("- Top-2 제한으로 필요한 문서들을 모두 검색하지 못함")
        print("- '파스텔' 키워드로만 검색하여 미술관 정보 누락 가능성")
        print("- 벡터 유사도만으로는 멀티홉 질문의 복잡성 처리 불가")
    
    print()
    print("#### 2️⃣ **맥락 이해 실패**") 
    print("**나이브 RAG의 한계:**")
    print("- '서울 가는데'라는 맥락으로 호텔임을 추론하지 못함")
    print("- '예약 변경'이라는 단서를 활용하지 못함")
    print("- 동음이의어 '파스텔' 중 어느 것인지 판단 불가")
    
    print()
    print("#### 3️⃣ **정보 연결 실패**")
    print("**멀티홉 추론 부족:**")
    print("- 파스텔 호텔 정보 + 미술관 정보를 독립적으로 처리")
    print("- 두 정보를 종합한 통합 답변 생성 불가")
    print("- 단순 검색 결과 나열에 그침")
    
    print()
    print("#### 4️⃣ **STRICT 프롬프트의 영향**")
    print("**일반지식 차단 효과:**")
    print("- LLM의 배경지식 활용 불가")
    print("- 컨텍스트에 없는 정보는 절대 추론하지 않음")
    print("- 진짜 RAG 시스템의 한계가 그대로 노출됨")

### 📊 나이브 RAG vs 사람의 추론 과정 비교

def compare_reasoning_process():
    """나이브 RAG와 사람의 추론 과정 비교"""
    print()
    print("### 📊 **추론 과정 비교**")
    print("---")
    print()
    
    print("| 단계 | 👤 **사람의 추론** | 🤖 **나이브 RAG** |")
    print("|------|-------------------|-------------------|")
    print("| **1. 질문 이해** | '서울 가는데' → 여행/숙박 상황<br/>'예약 변경' → 호텔 가능성 높음 | '파스텔' 키워드만 추출<br/>맥락 무시 |")
    print("| **2. 엔티티 해결** | 파스텔 = 호텔로 판단<br/>(맥락 기반 추론) | 파스텔 = 모호함<br/>(동음이의어 미해결) |")
    print("| **3. 정보 수집** | 호텔 정보 + 미술관 정보<br/>둘 다 필요하다고 판단 | 유사도 높은 문서 2개만<br/>편향적 검색 |")
    print("| **4. 답변 생성** | 두 정보를 종합하여<br/>완전한 답변 제공 | 검색된 정보만으로<br/>부분적 답변 |")
    
    print()
    print("**🎯 핵심 차이점:**")
    print("- **사람**: 맥락을 이해하고 추론하여 정보를 연결")  
    print("- **나이브 RAG**: 단순 키워드 매칭과 유사도 검색에 의존")

# 분석 함수들 실행
show_correct_answer()
analyze_failure_detailed(test_result)
compare_reasoning_process()

## 5. 핵심 교훈 및 Advanced RAG 예고 (10분)

### 🎓 핵심 교훈

#### ✅ **오늘 배운 것들**
1. **🤷 동음이의어 문제**: "파스텔" 하나만으로도 RAG는 혼란에 빠짐
2. **🧩 멀티홉 추론 한계**: 여러 문서 정보를 연결하는 능력 부족  
3. **📍 맥락 이해 실패**: "서울 가는데"라는 중요한 단서를 놓침
4. **🔒 STRICT 프롬프트 효과**: LLM 일반지식 차단으로 진짜 RAG 한계 노출

#### ⚠️ **나이브 RAG의 근본적 문제**
```
💭 사용자 의도: "여행용 숙소 예약 변경 + 문화생활 추천"
🤖 나이브 RAG: "파스텔이라는 키워드로 유사한 문서 2개 검색"
```

**결과**: 사용자가 원하는 통합적 답변이 아닌 단편적 정보만 제공

### 🚀 Advanced RAG로 가는 길

#### 📝 **다음 실습에서 해결할 문제들**

- 🎯 **쿼리 분해**: "파스텔 예약 변경" + "미술관 추천" → 2개 하위 질문
- 🔍 **엔티티 명확화**: 맥락을 통한 "파스텔 = 호텔" 추론
- 📊 **결과**: 동음이의어 문제 해결!

- 🏷️ **메타데이터 활용**: type="hotel" 필터링으로 정확한 문서 선택
- 📍 **지역 기반 검색**: location 정보로 관련 문화시설 추천
- 📊 **결과**: 불필요한 문서 제거, 정확도 향상!

- 🔄 **하이브리드 검색**: 키워드 + 벡터 검색으로 recall 향상  
- 📈 **리랭킹**: 질문과의 관련성으로 문서 재정렬
- 📊 **결과**: 멀티홉 정보 수집 성공!

### 💡 **실무 적용 포인트**

#### 🎯 **언제 Advanced RAG가 필요한가?**
- ✅ 사용자 질문이 모호하거나 복합적일 때
- ✅ 도메인별 전문 지식이 필요할 때  
- ✅ 여러 문서의 정보를 종합해야 할 때
- ✅ 높은 정확도가 요구되는 비즈니스 상황

#### ⚡ **성능 vs 복잡도 트레이드오프**
```
나이브 RAG    ←→    Advanced RAG
빠름, 단순     ←→    정확, 복잡
```

### 🎁 **직접 체험해보기**

파스텔 관련 다른 질문들로 나이브 RAG의 한계를 더 체험해보세요!

```python
# 예시 질문들
naive_rag({"query": "파스텔 주말 이벤트 있나요?"})
naive_rag({"query": "파스텔 근처에 뭐가 있어요?"})  
naive_rag({"query": "파스텔 예약하고 문화생활도 하고 싶어요"})
```

---

## 🎉 **다음 시간 예고**

**Advanced Query Refinement**로 파스텔의 정체성을 명확히 밝혀내고, 완벽한 멀티홉 답변을 만들어보겠습니다! 

*"나이브 RAG가 놓친 것들을 하나씩 해결해나가는 여정이 시작됩니다!"* 🚀

In [ ]:
# 🎉 실습 완료!
print("🎉 파스텔 나이브 RAG 분석 완료!")
print("다음 실습에서는 이 모든 문제들을 Advanced RAG 기법으로 해결해보겠습니다!")
print()
print("💡 지금까지 배운 것:")
print("  • 동음이의어 문제의 심각성")
print("  • 멀티홉 추론의 어려움") 
print("  • 나이브 RAG의 근본적 한계")
print()
print("🚀 다음에 배울 것:")
print("  • 쿼리 분해와 엔티티 명확화")
print("  • 메타데이터 필터링")
print("  • 하이브리드 검색과 리랭킹")